<a href="https://colab.research.google.com/github/JandreiSS/Analise-Fundamentalista/blob/master/DataFrame_bolsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import string
import requests
import warnings
warnings.filterwarnings('ignore')
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [3]:
url = 'http://fundamentus.com.br/resultado.php'

In [4]:
r = requests.get(url, headers=header)

In [5]:
df = pd.read_html(r.text)[0]

In [6]:
df = df.loc[:,['Papel']]

In [8]:
def get(url, timeout):
    return requests.get(url, headers=header, timeout = timeout)

In [9]:
def requestUrls(urls, timeout = 5):
    with ThreadPoolExecutor(max_workers = 3) as executor:
        agenda = { executor.submit(get, url, timeout): url for url in urls }

        for tarefa in as_completed(agenda):     
            try:
                conteudo = tarefa.result()
            except Exception as e:
                print ("Não foi possível fazer a requisição! \n{}".format(e))
            else:
                yield conteudo

In [12]:
urls = 'https://www.fundamentus.com.br/detalhes.php?papel=' + df['Papel'].values

In [13]:
requisicoes = requestUrls(urls, timeout=30)

In [14]:
lista_empresas = []

In [15]:
for requisicao in requisicoes:
    codigo = requisicao.status_code
    url = requisicao.url
    conteudo = requisicao.content
    lista_empresas += [conteudo]
    print ("{}: {}".format(codigo, url))

200: https://www.fundamentus.com.br/detalhes.php?papel=CLAN3
200: https://www.fundamentus.com.br/detalhes.php?papel=VNET3
200: https://www.fundamentus.com.br/detalhes.php?papel=CFLU4
200: https://www.fundamentus.com.br/detalhes.php?papel=CLSC6
200: https://www.fundamentus.com.br/detalhes.php?papel=CSTB3
200: https://www.fundamentus.com.br/detalhes.php?papel=CSTB4
200: https://www.fundamentus.com.br/detalhes.php?papel=MNSA3
200: https://www.fundamentus.com.br/detalhes.php?papel=IVTT3
200: https://www.fundamentus.com.br/detalhes.php?papel=MNSA4
200: https://www.fundamentus.com.br/detalhes.php?papel=PMET3
200: https://www.fundamentus.com.br/detalhes.php?papel=POPR4
200: https://www.fundamentus.com.br/detalhes.php?papel=PORP4
200: https://www.fundamentus.com.br/detalhes.php?papel=NORD3
200: https://www.fundamentus.com.br/detalhes.php?papel=DASA3
200: https://www.fundamentus.com.br/detalhes.php?papel=OMGE3
200: https://www.fundamentus.com.br/detalhes.php?papel=SQIA3
200: https://www.fundame

In [16]:
len(lista_empresas)

887

In [17]:
dados = pd.read_html(lista_empresas[0])

In [20]:
dados[0]

,0,1,2,3
0,?Papel,CLAN3,?Cotação,000
1,?Tipo,ON,?Data últ cot,-
2,?Empresa,CLARION ON,?Min 52 sem,000
3,?Setor,Alimentos,?Max 52 sem,000
4,?Subsetor,Grãos e Derivados,?Vol $ méd (2m),0


In [23]:
dados[0].to_csv('dados.csv', index=False)